# Learning Analytics & Student Performance Prediction

Complete tutorial on analyzing student data and predicting academic outcomes.

## Dataset

30 students with:
- **Demographics**: Gender, Age
- **Behavioral**: Study hours per week, Attendance %
- **Performance**: Assignment, Midterm, Final scores
- **Outcome**: Pass/Fail (60% threshold)

## Methods
- Exploratory data analysis
- Correlation analysis
- Logistic regression for pass/fail prediction
- Feature importance
- Performance visualization

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, classification_report, confusion_matrix, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore")

plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("Set2")
%matplotlib inline

print("✓ Setup complete")

## 1. Load and Explore Data

In [ ]:
# Load data
df = pd.read_csv("sample_student_data.csv")

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {', '.join(df.columns)}")
print(f"\nMissing values: {df.isnull().sum().sum()}")

df.head(10)

In [ ]:
# Summary statistics
print("Summary Statistics:")
print(df.describe())

print(f"\nPass Rate: {df['passed'].mean() * 100:.1f}%")
print(f"Passed: {df['passed'].sum()} students")
print(f"Failed: {(~df['passed'].astype(bool)).sum()} students")

## 2. Visualize Distributions

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

numeric_cols = [
    "study_hours",
    "attendance",
    "assignment_score",
    "midterm_score",
    "final_score",
    "age",
]

for idx, col in enumerate(numeric_cols):
    axes[idx].hist(df[col], bins=10, edgecolor="black", alpha=0.7)
    axes[idx].axvline(
        df[col].mean(),
        color="red",
        linestyle="--",
        linewidth=2,
        label=f"Mean: {df[col].mean():.1f}",
    )
    axes[idx].set_title(f"{col.replace('_', ' ').title()} Distribution", fontweight="bold")
    axes[idx].set_xlabel(col.replace("_", " ").title())
    axes[idx].set_ylabel("Frequency")
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Pass/Fail Analysis

In [ ]:
# Compare passed vs failed students
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

compare_vars = ["study_hours", "attendance", "assignment_score", "midterm_score"]

for idx, var in enumerate(compare_vars):
    passed = df[df["passed"] == 1][var]
    failed = df[df["passed"] == 0][var]

    axes[idx].boxplot([passed, failed], labels=["Passed", "Failed"])
    axes[idx].set_title(f"{var.replace('_', ' ').title()} by Outcome", fontweight="bold")
    axes[idx].set_ylabel(var.replace("_", " ").title())
    axes[idx].grid(True, alpha=0.3, axis="y")

    # Statistical test
    t_stat, p_val = stats.ttest_ind(passed, failed)
    sig = "***" if p_val < 0.001 else ("**" if p_val < 0.01 else ("*" if p_val < 0.05 else "ns"))
    axes[idx].text(
        0.5,
        0.95,
        f"p = {p_val:.4f} {sig}",
        transform=axes[idx].transAxes,
        ha="center",
        va="top",
        bbox={"boxstyle": "round", "facecolor": "wheat", "alpha": 0.5},
    )

plt.tight_layout()
plt.show()

## 4. Correlation Analysis

In [ ]:
# Select numeric columns for correlation
numeric_df = df[
    [
        "age",
        "study_hours",
        "attendance",
        "assignment_score",
        "midterm_score",
        "final_score",
        "passed",
    ]
]

correlation = numeric_df.corr()

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(
    correlation,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    center=0,
    square=True,
    ax=ax,
    cbar_kws={"label": "Correlation"},
)
ax.set_title("Student Performance Correlation Matrix", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

print("\nStrongest Predictors of Passing:")
pass_corr = correlation["passed"].sort_values(ascending=False)[1:]
for var, corr in pass_corr.items():
    print(f"  {var.replace('_', ' ').title()}: {corr:.3f}")

## 5. Feature Relationships

In [ ]:
# Scatter plots with regression lines
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

relationships = [
    ("study_hours", "final_score"),
    ("attendance", "final_score"),
    ("assignment_score", "final_score"),
    ("midterm_score", "final_score"),
]

for idx, (x_var, y_var) in enumerate(relationships):
    # Color by pass/fail
    passed = df[df["passed"] == 1]
    failed = df[df["passed"] == 0]

    axes[idx].scatter(passed[x_var], passed[y_var], c="green", alpha=0.6, s=100, label="Passed")
    axes[idx].scatter(failed[x_var], failed[y_var], c="red", alpha=0.6, s=100, label="Failed")

    # Regression line
    z = np.polyfit(df[x_var], df[y_var], 1)
    p = np.poly1d(z)
    x_line = np.linspace(df[x_var].min(), df[x_var].max(), 100)
    axes[idx].plot(x_line, p(x_line), "b--", alpha=0.8, linewidth=2)

    # Correlation
    r = df[[x_var, y_var]].corr().iloc[0, 1]
    axes[idx].text(
        0.05,
        0.95,
        f"r = {r:.3f}",
        transform=axes[idx].transAxes,
        va="top",
        bbox={"boxstyle": "round", "facecolor": "wheat", "alpha": 0.5},
    )

    axes[idx].set_xlabel(x_var.replace("_", " ").title(), fontsize=11)
    axes[idx].set_ylabel(y_var.replace("_", " ").title(), fontsize=11)
    axes[idx].set_title(
        f"{x_var.replace('_', ' ').title()} vs {y_var.replace('_', ' ').title()}", fontweight="bold"
    )
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Logistic Regression Model

In [ ]:
# Prepare features
feature_cols = ["study_hours", "attendance", "assignment_score", "midterm_score"]
X = df[feature_cols]
y = df["passed"]

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train model
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_scaled, y_train)

# Predictions
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

print("Model Training Complete!")
print(f"\nTraining set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")
print(f"\nTraining accuracy: {model.score(X_train_scaled, y_train):.3f}")
print(f"Test accuracy: {model.score(X_test_scaled, y_test):.3f}")

In [ ]:
# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Failed", "Passed"]))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    ax=ax,
    xticklabels=["Failed", "Passed"],
    yticklabels=["Failed", "Passed"],
)
ax.set_title("Confusion Matrix", fontsize=14, fontweight="bold")
ax.set_ylabel("True Label", fontsize=12)
ax.set_xlabel("Predicted Label", fontsize=12)
plt.tight_layout()
plt.show()

## 7. Feature Importance

In [ ]:
# Feature coefficients
coefficients = pd.DataFrame(
    {
        "Feature": feature_cols,
        "Coefficient": model.coef_[0],
        "Abs_Coefficient": np.abs(model.coef_[0]),
    }
).sort_values("Abs_Coefficient", ascending=False)

print("Feature Importance (Logistic Regression Coefficients):")
print(coefficients[["Feature", "Coefficient"]].to_string(index=False))

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
colors = ["green" if c > 0 else "red" for c in coefficients["Coefficient"]]
ax.barh(coefficients["Feature"], coefficients["Coefficient"], color=colors, alpha=0.7)
ax.axvline(0, color="black", linestyle="--", linewidth=1)
ax.set_title(
    "Feature Importance (Higher = More Important for Passing)", fontsize=12, fontweight="bold"
)
ax.set_xlabel("Coefficient")
ax.grid(True, alpha=0.3, axis="x")
plt.tight_layout()
plt.show()

## 8. ROC Curve

In [ ]:
# ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

fig, ax = plt.subplots(figsize=(8, 8))
ax.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (AUC = {roc_auc:.2f})")
ax.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--", label="Random Classifier")
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel("False Positive Rate", fontsize=12)
ax.set_ylabel("True Positive Rate", fontsize=12)
ax.set_title("Receiver Operating Characteristic (ROC) Curve", fontsize=14, fontweight="bold")
ax.legend(loc="lower right", fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nAUC Score: {roc_auc:.3f}")
print(f"Interpretation: {'Excellent' if roc_auc > 0.9 else ('Good' if roc_auc > 0.8 else 'Fair')}")

## 9. Summary Report

In [ ]:
# Generate summary
summary = pd.DataFrame(
    {
        "Metric": [
            "Total Students",
            "Pass Rate",
            "Model Accuracy",
            "AUC Score",
            "Top Predictor",
            "Mean Study Hours",
            "Mean Attendance",
        ],
        "Value": [
            len(df),
            f"{df['passed'].mean() * 100:.1f}%",
            f"{model.score(X_test_scaled, y_test):.3f}",
            f"{roc_auc:.3f}",
            coefficients.iloc[0]["Feature"],
            f"{df['study_hours'].mean():.1f} hrs/week",
            f"{df['attendance'].mean():.1f}%",
        ],
    }
)

print("=" * 70)
print("LEARNING ANALYTICS SUMMARY")
print("=" * 70)
print(summary.to_string(index=False))
print("=" * 70)

# Save
summary.to_csv("learning_analytics_summary.csv", index=False)
print("\n✓ Summary saved to learning_analytics_summary.csv")

## Key Findings

### Student Performance
- **Pass rate**: ~83% of students passed
- Strong correlation between **study hours** and final scores
- **Attendance** is a critical factor for success

### Prediction Model
- Logistic regression achieves high accuracy
- **Top predictors** (in order):
  1. Midterm score
  2. Assignment score
  3. Study hours
  4. Attendance

### Interventions
- Students with < 8 study hours/week at risk
- Attendance below 70% strongly predicts failure
- Early midterm performance is best predictor

## Next Steps

1. **Real-time monitoring**: Track at-risk students early
2. **Feature engineering**: Add engagement metrics (clicks, time-on-task)
3. **Advanced models**: Random Forest, XGBoost
4. **A/B testing**: Test intervention strategies
5. **Longitudinal analysis**: Track cohorts over time

## Resources

- [Educational Data Mining](https://educationaldatamining.org/)
- [Learning Analytics](https://www.solaresearch.org/)
- [Scikit-learn Documentation](https://scikit-learn.org/)